In [2]:
# 수능 국어 문제 자동 채점 시스템 (OpenAI v1.0+ 호환)
# 주피터 노트북 셀 단위로 실행 가능하도록 구성

# ==================== Cell 1: 라이브러리 import 및 설정 ====================
from openai import OpenAI  # 새로운 방식
import pandas as pd
import json
import re
import time
from typing import List, Dict, Any, Tuple, Optional
import os

# OpenAI 클라이언트 초기화
# API 키는 환경변수에서 읽어오거나 직접 설정
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")  # 실제 API 키로 변경하거나 환경변수 OPENAI_API_KEY 사용
)

# 환경변수에서 API 키를 읽어오는 경우:
# client = OpenAI()  # OPENAI_API_KEY 환경변수에서 자동으로 읽음

# ==================== Cell 2: 데이터 로드 및 전처리 ====================
# JSON 파일에서 문제 데이터 로드
def load_problems(file_path: str) -> List[Dict]:
    """문제 데이터를 JSON 파일에서 로드"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return []
    except json.JSONDecodeError:
        print("JSON 파일 형식이 올바르지 않습니다.")
        return []

# 샘플 데이터 (실제로는 파일에서 로드)
sample_data = [
    {
        "id": "2023_11_KICE_1-3",
        "paragraph": "사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 '소통의 즐거움'이 있다...",
        "type": 0,
        "problems": [
            {
                "question": "윗글의 내용과 일치하지 않는 것은?",
                "choices": [
                    "같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다.",
                    "다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다",
                    "독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다.",
                    "독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.",
                    "독자는 책을 읽을 때 자신이 속한 사회나 시대의 영향을 받으며 필자와 간접적으로 대화한다"
                ],
                "answer": 4,
                "score": 2
            }
        ]
    }
]

# 실제 사용할 때는 아래 라인을 사용
all_problems = load_problems('./2023_11_KICE.json')
# all_problems = sample_data

print(f"로드된 문제 세트 수: {len(all_problems)}")

# ==================== Cell 3: 모델 및 프롬프트 정의 ====================
# 사용할 모델들
models = ["gpt-4o-mini", "gpt-4o"]

# 프롬프트 템플릿들
prompts = {
    "zero_shot": """다음 지문을 읽고 문제를 풀어주세요.

지문:
{paragraph}

문제: {question}
{question_plus}

선택지:
{choices}

정답 번호만 답해주세요 (1~5 중 하나).""",

    "negative": """다음 지문을 읽고 문제를 풀어주세요.

주의사항:
- 추측하지 말고 지문의 내용에 근거하여 답해주세요
- 확실하지 않은 답은 피해주세요
- 선택지를 꼼꼼히 비교 분석해주세요

지문:
{paragraph}

문제: {question}
{question_plus}

선택지:
{choices}

정답 번호만 답해주세요 (1~5 중 하나).""",

    "function_calling": {
        "question_plus": """다음은 추가 지문이 있는 문제입니다. 본문과 추가 지문을 모두 참고하여 답해주세요.

지문:
{paragraph}

문제: {question}

추가 지문:
{question_plus}

선택지:
{choices}

정답 번호만 답해주세요 (1~5 중 하나).""",

        "vocabulary": """다음은 어휘 및 문법 문제입니다. 정확한 어휘의 의미와 문법 규칙을 적용하여 답해주세요.

지문:
{paragraph}

문제: {question}
{question_plus}

선택지:
{choices}

정답 번호만 답해주세요 (1~5 중 하나).""",

        "comprehension": """다음은 내용 파악 문제입니다. 지문의 내용을 정확히 이해하고 답해주세요.

지문:
{paragraph}

문제: {question}
{question_plus}

선택지:
{choices}

정답 번호만 답해주세요 (1~5 중 하나).""",

        "inference": """다음은 추론 문제입니다. 지문의 내용을 바탕으로 논리적으로 추론하여 답해주세요.

지문:
{paragraph}

문제: {question}
{question_plus}

선택지:
{choices}

정답 번호만 답해주세요 (1~5 중 하나)."""
    }
}

# 전역 변수 (현재 사용 중인 모델과 프롬프트)
current_model = None
current_prompt_type = None

print("모델 및 프롬프트 설정 완료")

# ==================== Cell 4: 유틸리티 함수들 ====================
def get_question_type_by_model(question: str, question_plus: Optional[str] = None) -> str:
    """GPT 모델을 통해 문제 유형을 분류"""
    
    classification_prompt = f"""다음 문제의 유형을 분류해주세요.

문제: {question}
추가내용: {question_plus if question_plus else "없음"}

다음 중 하나로 분류해주세요:
1. question_plus - 추가 지문이나 자료가 있는 문제
2. vocabulary - 어휘나 문법 관련 문제  
3. comprehension - 내용 파악 문제
4. inference - 내용 기반 추론 문제

분류 결과만 답해주세요 (question_plus, vocabulary, comprehension, inference 중 하나):"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # 분류용으로는 저렴한 모델 사용
            messages=[{"role": "user", "content": classification_prompt}],
            max_tokens=10,
            temperature=0
        )
        
        result = response.choices[0].message.content.strip().lower()
        
        # 결과 검증 및 기본값 설정
        valid_types = ["question_plus", "vocabulary", "comprehension", "inference"]
        for valid_type in valid_types:
            if valid_type in result:
                return valid_type
        
        # 기본값: question_plus가 있으면 question_plus, 없으면 comprehension
        return "question_plus" if question_plus else "comprehension"
        
    except Exception as e:
        print(f"문제 유형 분류 중 오류 발생: {e}")
        return "question_plus" if question_plus else "comprehension"


def extract_answer_from_response(response: str, choices: List[str]) -> int:
    """모델 응답에서 정답 번호를 추출"""
    
    # 1~5 숫자 패턴 찾기
    number_patterns = [
        r'(?:정답은?\s*)?([1-5])(?:번)?',
        r'([1-5])',
        r'(?:답|선택):\s*([1-5])',
    ]
    
    for pattern in number_patterns:
        matches = re.findall(pattern, response)
        if matches:
            try:
                answer_num = int(matches[0])
                if 1 <= answer_num <= 5:
                    return answer_num
            except ValueError:
                continue
    
    # 선택지 텍스트 매칭 시도
    for i, choice in enumerate(choices):
        # 선택지의 주요 키워드가 응답에 포함되어 있는지 확인
        choice_keywords = choice.split()[:3]  # 처음 3단어만 사용
        if len(choice_keywords) >= 2:
            keyword_phrase = ' '.join(choice_keywords)
            if keyword_phrase in response:
                return i + 1
    
    # 기본값 (추출 실패 시)
    print(f"답안 추출 실패. 응답: {response[:100]}...")
    return 1


def format_choices(choices: List[str]) -> str:
    """선택지를 문자열로 포맷팅"""
    formatted = []
    for i, choice in enumerate(choices, 1):
        formatted.append(f"{i}. {choice}")
    return '\n'.join(formatted)


def call_openai_api(prompt: str, model: str, max_retries: int = 3) -> str:
    """OpenAI API 호출 (재시도 로직 포함) - 새로운 방식"""
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=0
            )
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            print(f"API 호출 실패 (시도 {attempt + 1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # 지수 백오프
            else:
                return "API 호출 실패"

print("유틸리티 함수 정의 완료")

# ==================== Cell 5: 핵심 prediction 함수 ====================
def prediction(problems: List[Dict]) -> List[int]:
    """
    주어진 문제들에 대해 현재 설정된 모델과 프롬프트로 예측 수행
    
    Args:
        problems: 문제 리스트 (각 문제는 question, choices, answer, score 포함)
    
    Returns:
        List[int]: 각 문제에 대한 예측 답안 번호 (1~5)
    """
    global current_model, current_prompt_type
    
    if not current_model or not current_prompt_type:
        raise ValueError("current_model과 current_prompt_type이 설정되지 않았습니다.")
    
    predictions = []
    
    for i, problem in enumerate(problems):
        question = problem['question']
        choices = problem['choices']
        question_plus = problem.get('question_plus', '')
        
        # 프롬프트 선택 및 구성
        if current_prompt_type == "function_calling":
            # 모델이 문제 유형을 판단
            question_type = get_question_type_by_model(question, question_plus)
            prompt_template = prompts[current_prompt_type][question_type]
            print(f"  문제 {i+1}: Function Calling - {question_type} 타입")
        else:
            prompt_template = prompts[current_prompt_type]
        
        # 프롬프트 포맷팅
        formatted_prompt = prompt_template.format(
            paragraph=current_paragraph,  # 전역 변수에서 참조
            question=question,
            question_plus=f"\n추가 내용:\n{question_plus}" if question_plus else "",
            choices=format_choices(choices)
        )
        
        # API 호출
        response = call_openai_api(formatted_prompt, current_model)
        
        # 답안 추출
        predicted_answer = extract_answer_from_response(response, choices)
        predictions.append(predicted_answer)
        
        print(f"  문제 {i+1} 응답: {response[:50]}... → 예측: {predicted_answer}")
    
    return predictions

print("prediction 함수 정의 완료")

# ==================== Cell 6: 정답 및 배점 데이터 준비 ====================
# 수능 정답 저장 배열 생성
correct_answers = []
scores = []
problem_info = []  # 문제 정보 저장 (디버깅용)

for problem_set in all_problems:
    for problem in problem_set['problems']:
        correct_answers.append(problem['answer'])
        scores.append(problem['score'])
        problem_info.append({
            'id': problem_set['id'],
            'question': problem['question'][:50] + "..." if len(problem['question']) > 50 else problem['question']
        })

print(f"총 문제 수: {len(correct_answers)}")
print(f"총 배점: {sum(scores)}점")

# ==================== Cell 7: 컴비네이션 정의 및 실행 준비 ====================
# 각 프롬프트별 컴비네이션 정의
combinations = [
    ("gpt-4o-mini", "zero_shot"),
    ("gpt-4o-mini", "negative"), 
    ("gpt-4o-mini", "function_calling"),
    ("gpt-4o", "zero_shot"),
    ("gpt-4o", "negative"),
    ("gpt-4o", "function_calling")
]

# 컴비네이션 별 점수 저장 딕셔너리
combination_scores = {}
detailed_results = {}  # 상세 결과 저장

print(f"실행할 조합 수: {len(combinations)}")
for i, (model, prompt) in enumerate(combinations, 1):
    print(f"{i}. {model} + {prompt}")

# ==================== Cell 8: solve_problem 함수 정의 ====================
def solve_problem(all_problems: List[Dict], combination: Tuple[str, str]) -> int:
    """
    주어진 문제들을 특정 모델-프롬프트 조합으로 해결
    
    Returns:
        int: 총 획득 점수
    """
    global current_model, current_prompt_type, current_paragraph
    
    model, prompt_type = combination
    current_model, current_prompt_type = model, prompt_type
    
    print(f"\n{'='*60}")
    print(f"실행 중: {model} + {prompt_type}")
    print(f"{'='*60}")
    
    total_score = 0
    problem_idx = 0
    detailed_result = []
    
    for problem_set in all_problems:
        current_paragraph = problem_set['paragraph']
        problems = problem_set['problems']
        
        print(f"\n문제 세트 ID: {problem_set['id']}")
        print("-" * 40)
        
        # 모델 예측 수행
        model_predictions = prediction(problems)
        
        # 각 문제별 정답 비교
        for i, (pred, problem) in enumerate(zip(model_predictions, problems)):
            correct = problem['answer']
            score = problem['score']
            
            is_correct = (pred == correct)
            earned_score = score if is_correct else 0
            total_score += earned_score
            
            result_symbol = "✓" if is_correct else "✗"
            print(f"  {result_symbol} 문제 {i+1}: 예측={pred}, 정답={correct}, 점수={earned_score}/{score}")
            
            # 상세 결과 저장
            detailed_result.append({
                'problem_set_id': problem_set['id'],
                'problem_num': i + 1,
                'question': problem['question'][:100] + "..." if len(problem['question']) > 100 else problem['question'],
                'predicted': pred,
                'correct': correct,
                'is_correct': is_correct,
                'score': earned_score,
                'max_score': score
            })
            
            problem_idx += 1   
    accuracy = sum(1 for r in detailed_result if r['is_correct']) / len(detailed_result) * 100
    max_possible_score = sum(scores)
    
    print(f"\n{'='*60}")
    print(f"최종 결과: {model} + {prompt_type}")
    print(f"총 점수: {total_score}/{max_possible_score}점 ({total_score/max_possible_score*100:.1f}%)")
    print(f"정답률: {accuracy:.1f}% ({sum(1 for r in detailed_result if r['is_correct'])}/{len(detailed_result)})")
    print(f"{'='*60}")
    
    # 결과 저장
    combination_key = f"{model}_{prompt_type}"
    combination_scores[combination_key] = total_score
    detailed_results[combination_key] = detailed_result
    
    return total_score

print("solve_problem 함수 정의 완료")

# ==================== Cell 9: 메인 실행 루프 ====================
print("🚀 실험 시작!")
print(f"총 {len(combinations)}개 조합을 실행합니다.")

# 각 조합별로 실행
for i, combination in enumerate(combinations, 1):
    print(f"\n🔄 진행률: {i}/{len(combinations)}")
    solve_problem(all_problems, combination)
    
    # API 호출 제한을 고려한 대기 시간
    if i < len(combinations):
        print("⏳ 다음 조합 실행을 위해 잠시 대기...")
        time.sleep(2)

print("\n🎉 모든 실험 완료!")

# ==================== Cell 10: 결과 분석 및 시각화 ====================
def create_results_table(combination_scores: Dict[str, int]) -> pd.DataFrame:
    """결과를 테이블 형태로 정리"""
    
    # 데이터 재구성
    data = []
    for combination_key, score in combination_scores.items():
        model, prompt = combination_key.split('_', 1)
        data.append({
            'Model': model,
            'Prompt': prompt,
            'Score': score,
            'Max_Score': sum(scores),
            'Accuracy_%': round(score / sum(scores) * 100, 1)
        })
    
    df = pd.DataFrame(data)
    return df

def create_pivot_table(df: pd.DataFrame) -> pd.DataFrame:
    """피벗 테이블 생성 (모델 vs 프롬프트)"""
    pivot = df.pivot(index='Model', columns='Prompt', values='Score')
    return pivot

# 결과 테이블 생성
results_df = create_results_table(combination_scores)
pivot_df = create_pivot_table(results_df)

print("\n📊 전체 결과 요약")
print("=" * 80)
print(results_df.to_string(index=False))

print(f"\n📈 피벗 테이블 (점수)")
print("=" * 50)
print(pivot_df.to_string())

# 최고 성능 조합 찾기
best_combination = results_df.loc[results_df['Score'].idxmax()]
print(f"\n🏆 최고 성능 조합:")
print(f"   모델: {best_combination['Model']}")
print(f"   프롬프트: {best_combination['Prompt']}")
print(f"   점수: {best_combination['Score']}/{best_combination['Max_Score']}점")
print(f"   정확도: {best_combination['Accuracy_%']}%")

# ==================== Cell 11: 상세 분석 (선택적) ====================
def analyze_wrong_answers(detailed_results: Dict, combination_key: str):
    """특정 조합의 오답 분석"""
    
    results = detailed_results[combination_key]
    wrong_answers = [r for r in results if not r['is_correct']]
    
    print(f"\n🔍 {combination_key} 오답 분석")
    print(f"총 오답 수: {len(wrong_answers)}")
    print("-" * 60)
    
    for wrong in wrong_answers[:5]:  # 최대 5개만 표시
        print(f"문제: {wrong['question']}")
        print(f"예측: {wrong['predicted']}, 정답: {wrong['correct']}")
        print(f"배점: {wrong['max_score']}점")
        print("-" * 40)

# 최고 성능 조합의 오답 분석
best_key = f"{best_combination['Model']}_{best_combination['Prompt']}"
if best_key in detailed_results:
    analyze_wrong_answers(detailed_results, best_key)

print("\n✅ 분석 완료! 결과를 확인해주세요.")

로드된 문제 세트 수: 11
모델 및 프롬프트 설정 완료
유틸리티 함수 정의 완료
prediction 함수 정의 완료
총 문제 수: 45
총 배점: 100점
실행할 조합 수: 6
1. gpt-4o-mini + zero_shot
2. gpt-4o-mini + negative
3. gpt-4o-mini + function_calling
4. gpt-4o + zero_shot
5. gpt-4o + negative
6. gpt-4o + function_calling
solve_problem 함수 정의 완료
🚀 실험 시작!
총 6개 조합을 실행합니다.

🔄 진행률: 1/6

실행 중: gpt-4o-mini + zero_shot

문제 세트 ID: 2023_11_KICE_1-3
----------------------------------------
  문제 1 응답: 4... → 예측: 4
  문제 2 응답: 4... → 예측: 4
  문제 3 응답: 1... → 예측: 1
  ✓ 문제 1: 예측=4, 정답=4, 점수=2/2
  ✗ 문제 2: 예측=4, 정답=5, 점수=0/3
  ✓ 문제 3: 예측=1, 정답=1, 점수=2/2

문제 세트 ID: 2023_11_KICE_4-9
----------------------------------------
  문제 1 응답: 4... → 예측: 4
  문제 2 응답: 5... → 예측: 5
  문제 3 응답: 3... → 예측: 3
  문제 4 응답: 2... → 예측: 2
  문제 5 응답: 3... → 예측: 3
  문제 6 응답: 2... → 예측: 2
  ✓ 문제 1: 예측=4, 정답=4, 점수=2/2
  ✓ 문제 2: 예측=5, 정답=5, 점수=2/2
  ✓ 문제 3: 예측=3, 정답=3, 점수=2/2
  ✓ 문제 4: 예측=2, 정답=2, 점수=2/2
  ✗ 문제 5: 예측=3, 정답=5, 점수=0/3
  ✓ 문제 6: 예측=2, 정답=2, 점수=2/2

문제 세트 ID: 2023_11_KICE_10-13
-----